In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     
import torchvision
import torchvision.transforms as transforms

from models import *

global best_prec
use_gpu = torch.cuda.is_available()

In [2]:
batch_size = 128

In [3]:
normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
print_freq = 300

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [80, 120]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        


In [5]:
model_name = "resnet20_quant4"
model = resnet20_quant()

In [6]:
lr = 4e-2
weight_decay = 1e-4
epochs = 150
best_prec = 0

model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

fdir = '/home/pnataraj/private/ece284_saved_models/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)     

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)
    if best_prec > 90:
        break

Epoch: [0][0/391]	Time 0.813 (0.813)	Data 0.220 (0.220)	Loss 2.4892 (2.4892)	Prec 9.375% (9.375%)
Epoch: [0][300/391]	Time 0.043 (0.047)	Data 0.001 (0.002)	Loss 1.5429 (1.7049)	Prec 46.875% (35.821%)
Validation starts
Test: [0/79]	Time 0.140 (0.140)	Loss 1.3312 (1.3312)	Prec 52.344% (52.344%)
 * Prec 49.900% 
best acc: 49.900000
Epoch: [1][0/391]	Time 0.264 (0.264)	Data 0.213 (0.213)	Loss 1.1884 (1.1884)	Prec 57.812% (57.812%)
Epoch: [1][300/391]	Time 0.043 (0.044)	Data 0.002 (0.002)	Loss 1.2679 (1.1913)	Prec 60.156% (56.946%)
Validation starts
Test: [0/79]	Time 0.186 (0.186)	Loss 1.3814 (1.3814)	Prec 52.344% (52.344%)
 * Prec 53.130% 
best acc: 53.130000
Epoch: [2][0/391]	Time 0.273 (0.273)	Data 0.223 (0.223)	Loss 1.0689 (1.0689)	Prec 60.938% (60.938%)
Epoch: [2][300/391]	Time 0.047 (0.045)	Data 0.001 (0.002)	Loss 0.9337 (0.9758)	Prec 67.969% (65.140%)
Validation starts
Test: [0/79]	Time 0.181 (0.181)	Loss 0.8525 (0.8525)	Prec 71.094% (71.094%)
 * Prec 65.910% 
best acc: 65.910000
Epo

In [7]:
PATH = '/home/pnataraj/private/ece284_saved_models/resnet20_quant4/model_best.pth.tar'
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9030/10000 (90%)



In [8]:
#send an input and grap the value by using prehook like HW3
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()

for layer in model.modules():
    if isinstance(layer, torch.nn.Conv2d):        
        layer.register_forward_pre_hook(save_output)       ## Input for the module will be grapped 
####################################################

dataiter = iter(trainloader)
images, labels = next(dataiter)
images = images.cuda()
out = model(images)

In [9]:
w_bit = 4
weight_q = model.layer1[0].conv1.weight_q
w_alpha = model.layer1[0].conv1.weight_quant.wgt_alpha.data.item()
w_delta = w_alpha / (2 ** (w_bit-1) - 1)
weight_int = weight_q / w_delta
print(weight_int)

tensor([[[[-4.,  3.,  1.],
          [ 7., -1., -3.],
          [ 2.,  3.,  1.]],

         [[-0.,  2.,  0.],
          [-0., -1., -1.],
          [-3., -4., -3.]],

         [[-6., -1., -3.],
          [-5.,  3.,  1.],
          [ 2.,  5., -0.]],

         ...,

         [[-1., -0., -5.],
          [ 3.,  5., -2.],
          [ 3.,  0., -3.]],

         [[ 5.,  7.,  2.],
          [-7., -6., -4.],
          [-2., -0.,  1.]],

         [[ 3.,  4.,  0.],
          [-2., -7.,  2.],
          [-4.,  6.,  2.]]],


        [[[-1., -3., -1.],
          [-3.,  7., -1.],
          [ 2.,  0.,  3.]],

         [[-1., -2., -0.],
          [-3., -2., -1.],
          [ 1.,  0.,  2.]],

         [[-4., -1.,  1.],
          [-0.,  1., -4.],
          [ 1.,  3., -2.]],

         ...,

         [[-0.,  4.,  1.],
          [ 0., -1., -1.],
          [-1., -5., -5.]],

         [[-5., -6., -3.],
          [ 2., -2., -2.],
          [-1., -0.,  0.]],

         [[-1., -7., -2.],
          [-2.,  3., -4.],
 

In [10]:
x_bit = 4    
x = save_output.outputs[1][0]
x_alpha  = model.layer1[0].conv1.act_alpha.data.item() # 8.0
x_delta = x_alpha / (2 ** x_bit - 1)

act_quant_fn = act_quantization(x_bit)
x_q = act_quant_fn(x, x_alpha)

x_int = x_q / x_delta
print(x_int)

tensor([[[[ 0.0000,  7.0000,  7.0000,  ...,  7.0000,  7.0000,  0.0000],
          [ 0.0000,  7.0000,  4.0000,  ...,  1.0000,  2.0000,  0.0000],
          [ 0.0000,  0.0000, 12.0000,  ...,  9.0000,  5.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000, 15.0000,  ...,  0.0000,  8.0000, 14.0000],
          [ 0.0000,  0.0000, 15.0000,  ...,  1.0000,  8.0000, 15.0000],
          [ 0.0000,  0.0000, 15.0000,  ...,  3.0000,  9.0000, 15.0000]],

         [[ 6.0000,  5.0000,  5.0000,  ...,  5.0000,  5.0000,  0.0000],
          [ 2.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 2.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 2.0000,  0.0000,  4.0000,  ...,  1.0000,  0.0000,  2.0000],
          [ 2.0000,  0.0000,  2.0000,  ...,  0.0000,  0.0000,  3.0000],
          [ 0.0000,  0.0000,  2.0000,  ...,  2.0000,  4.0000,  4.0000]],

         [[ 0.0000,  4.0000,  4.0000,  ...,  4.0000,  4.0000, 15.0000],
          [ 0.0000,  6.0000,  

In [11]:
#  Check the recovered psum has similar value to the un-quantized original psum
#  (such as example 1 in W3S2)
conv_int = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

output_int =  conv_int(x_int)
output_recovered = output_int * x_delta * w_delta  # recover with x_delta and w_delta
print(output_recovered)

tensor([[[[ 1.7603e+01,  1.6333e+01,  2.4499e+00,  ...,  1.1433e+01,
            5.1720e+00, -1.7240e+00],
          [ 1.0525e+01, -1.7421e+01,  1.9055e+00,  ..., -3.2665e+01,
           -2.7493e+01, -9.0737e+00],
          [ 1.8873e+01,  4.1739e+00, -2.8128e+00,  ...,  2.8128e+00,
            3.6295e+00,  7.2589e+00],
          ...,
          [ 2.0597e+01, -4.8090e+00, -2.3319e+01,  ..., -1.8420e+01,
           -5.5349e+00,  1.0616e+01],
          [ 1.0707e+01, -2.6041e+01, -4.5368e-01,  ..., -6.4423e+00,
            2.3682e+01,  6.3516e+00],
          [ 2.6586e+01,  1.7694e+01,  4.7818e+01,  ...,  1.3520e+01,
            1.2431e+01,  2.7221e-01]],

         [[ 6.7145e+00, -1.3792e+01, -1.1977e+01,  ..., -8.6200e+00,
           -1.4518e+01, -1.9055e+01],
          [-9.9810e-01,  1.4972e+01,  2.7221e+00,  ...,  2.4953e+01,
            2.5860e+01,  1.1251e+01],
          [-5.3535e+00, -5.5349e+00, -9.8903e+00,  ..., -5.3535e+00,
            1.2703e+00, -7.2590e-01],
          ...,
     

In [12]:
conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
conv_ref.weight = model.layer1[0].conv1.weight_q

output_ref =  conv_ref(x)
print(output_ref)

tensor([[[[ 1.6845e+01,  1.8424e+01,  2.6131e+00,  ...,  1.1276e+01,
            6.1320e+00, -3.1428e+00],
          [ 1.1592e+01, -1.7665e+01,  6.6410e-01,  ..., -3.2464e+01,
           -2.6848e+01, -1.0149e+01],
          [ 2.0634e+01,  4.9903e+00, -3.3099e+00,  ...,  2.6158e+00,
            3.0775e+00,  7.2962e+00],
          ...,
          [ 3.2908e+01, -1.2732e+01, -2.5913e+01,  ..., -1.7750e+01,
           -4.9344e+00,  9.7403e+00],
          [ 1.9246e+01, -3.8230e+01,  6.5092e+00,  ..., -6.1206e+00,
            2.3472e+01,  5.6437e+00],
          [ 2.9471e+01,  6.0703e+00,  5.2223e+01,  ...,  1.4188e+01,
            1.1188e+01, -1.2095e+00]],

         [[ 8.1314e+00, -1.4277e+01, -1.1704e+01,  ..., -7.9669e+00,
           -1.3873e+01, -2.0383e+01],
          [-1.3673e+00,  1.6389e+01,  2.7147e+00,  ...,  2.4226e+01,
            2.5522e+01,  1.1710e+01],
          [-4.6326e+00, -6.7049e+00, -1.0332e+01,  ..., -4.8426e+00,
            2.1366e+00, -1.6652e-01],
          ...,
     

In [13]:
diff = abs(output_ref - output_recovered)
print(diff.mean())

tensor(1.3324, device='cuda:0', grad_fn=<MeanBackward0>)


In [14]:
# Switch bits to 2 in quant_layer.py
from models import *

In [17]:
model_name = "resnet20_quant2"
model = resnet20_quant()

In [18]:
lr = 4e-2
weight_decay = 1e-4
epochs = 200
best_prec = 0

model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

fdir = '/home/pnataraj/private/ece284_saved_models/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)     

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

    if best_prec > 90:
            break

Epoch: [0][0/391]	Time 0.204 (0.204)	Data 0.134 (0.134)	Loss 2.4191 (2.4191)	Prec 11.719% (11.719%)
Epoch: [0][300/391]	Time 0.043 (0.044)	Data 0.001 (0.002)	Loss 1.2167 (1.6921)	Prec 60.938% (36.028%)
Validation starts
Test: [0/79]	Time 0.181 (0.181)	Loss 1.4223 (1.4223)	Prec 50.000% (50.000%)
 * Prec 50.020% 
best acc: 50.020000
Epoch: [1][0/391]	Time 0.257 (0.257)	Data 0.209 (0.209)	Loss 1.2397 (1.2397)	Prec 54.688% (54.688%)
Epoch: [1][300/391]	Time 0.043 (0.044)	Data 0.001 (0.002)	Loss 1.3312 (1.1518)	Prec 57.031% (58.313%)
Validation starts
Test: [0/79]	Time 0.185 (0.185)	Loss 1.1842 (1.1842)	Prec 60.156% (60.156%)
 * Prec 56.430% 
best acc: 56.430000
Epoch: [2][0/391]	Time 0.243 (0.243)	Data 0.195 (0.195)	Loss 0.9798 (0.9798)	Prec 67.969% (67.969%)
Epoch: [2][300/391]	Time 0.043 (0.044)	Data 0.001 (0.002)	Loss 1.0320 (0.9470)	Prec 64.062% (66.274%)
Validation starts
Test: [0/79]	Time 0.195 (0.195)	Loss 0.9252 (0.9252)	Prec 66.406% (66.406%)
 * Prec 66.470% 
best acc: 66.470000
E

In [19]:
PATH = '/home/pnataraj/private/ece284_saved_models/resnet20_quant2/model_best.pth.tar'
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9017/10000 (90%)

